In [1]:
import json
import os
from glob import glob
from tqdm import tqdm
from collections import defaultdict
from transformers import BertTokenizer
import pickle
import argparse

2022-01-19 11:40:46.954849: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-01-19 11:40:46.954934: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
indir = os.path.join('data','preprocessed')
outdir = os.path.join('data','preprocessed')
files = sorted(glob(os.path.join(indir, '*_tokenized.jsonlist')))
index_file = os.path.join(indir,'cofea_full_index.dict')
target_file = os.path.join(outdir,'target_word_index.dict')
word_file = os.path.join('data','constitution_words.txt')
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [3]:
# tokenize key words and phrases
with open(word_file, 'r',encoding = 'utf-8') as f:
    target_words = f.read().splitlines()
target_words = [tokenizer.tokenize(x) for x in target_words]
target_words_cleaned = []
for x in target_words:
    # rejoin into concatenated words
    rejoined_pieces = []
    for p_i, piece in enumerate(x):
        if p_i == 0:
            rejoined_pieces.append(piece)
        elif piece.startswith('##'):
            rejoined_pieces[-1] += piece
        else:
            rejoined_pieces.append(piece)
    target_words_cleaned.append(rejoined_pieces)

In [4]:
# get index
with open(index_file,'rb') as f:
    index = pickle.load(f)

In [5]:
target_index = {}
for word in tqdm(target_words_cleaned):
    if len(word) == 1:
        # just one word
        target_index[word[0]]=index.get(word[0],set())

    elif len(word) > 1:
        # we have a phrase
        phrase_indexes = []
        start = index.get(word[0])
        for f_id,doc_id,doc_index in start:
            match = False
            for x,piece in enumerate(word[1:]):
                if (f_id,doc_id,doc_index+x+1) in index.get(piece,set()):  
                    match = True
                else:
                    match = False
                    break

            if match:
                phrase_indexes.append((f_id,doc_id,doc_index))
        target_index[' '.join(word)] = set(phrase_indexes)


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1056/1056 [00:00<00:00, 1544.46it/s]


In [6]:
with open(target_file,'wb') as f:
    pickle.dump(target_index,file=f)